<a href="https://colab.research.google.com/github/41371125h-chinrouzhen/114-1-PL/blob/main/HW4_Github%E6%8A%80%E8%A1%93%E8%B6%A8%E5%8B%A2%E7%88%AC%E8%9F%B2%2BAI%E6%96%87%E5%AD%97%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Github技術趨勢爬蟲+AI文字分析（作業四）**
目標：呼叫 API → 做詞數與關鍵字計數 → 爬蟲結果寫入 → 輸出前 N 熱詞 → 回寫統計表。

AI ：使用者可以從 Top 5 熱詞中任選一個，AI產出 5 句洞察摘要 + 一段 120 字結論。

補充：
選擇GitHub作爲資料庫是最初想在104 或 Dcard 等網站存在嚴重反爬機制，Gemini便建議我直接選擇GitHub，因爲它有公開且穩定的API。


In [34]:
# 安裝
!pip install --upgrade google-generativeai
!pip install pygsheets gspread google-auth-oauthlib beautifulsoup4 jieba scikit-learn gradio pandas oauth2client lxml
print("安裝成功")

In [38]:
import pygsheets
from google.colab import files
import google.auth
import requests
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import google.generativeai as genai
import gradio as gr
import re
import json
from datetime import datetime
import os

JSON_KEY_FILE_PATH = "/content/gen-lang-client-0156076368-e711a064f70d.json"

try:
    if not os.path.exists(JSON_KEY_FILE_PATH):
        raise FileNotFoundError("找不到指定的 JSON 檔案。")

    gc = pygsheets.authorize(service_account_file=JSON_KEY_FILE_PATH) # <<< 修正：pygsheets 授權

except FileNotFoundError:
    print(f"'{JSON_KEY_FILE_PATH}'找不到檔案。")
except Exception as e:
    print(f"帳戶金鑰驗證失敗：{e}")
print("驗證成功")

驗證成功


In [39]:
GOOGLE_SHEET_NAME = "爬蟲資料"

from google.colab import userdata
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')

except userdata.SecretNotFoundError as e:
    print(f"找不到密鑰 '{e.name}'。")
    raise e

try:
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel('models/gemini-2.5-flash')
except Exception as e:
    print(f"Gemini API 設定失敗：{e}")

GITHUB_HEADERS = {
    'Authorization': f'token {GITHUB_TOKEN}',
    'Accept': 'application/vnd.github.v3+json'
}

In [41]:
def scrape_github_bios(search_location="Taiwan", num_users=50):
    per_page = min(num_users, 100)
    search_url = f"https://api.github.com/search/users?q=location:{search_location}&per_page={per_page}"
    try:
        response_search = requests.get(search_url, headers=GITHUB_HEADERS)
        response_search.raise_for_status()
        users = response_search.json().get('items', [])
        if not users:
            print("爬取失敗：找不到任何用戶。")
            return pd.DataFrame()
        print(f"成功搜尋到 {len(users)} 位用戶")
        data = []
        for user in users[:num_users]:
            user_url = user['url']
            try:
                response_user = requests.get(user_url, headers=GITHUB_HEADERS)
                response_user.raise_for_status()
                user_data = response_user.json()
                bio = user_data.get('bio')
                if bio and bio.strip():
                    data.append({
                        'username': user_data.get('login'),
                        'name': user_data.get('name'),
                        'bio': bio,
                        'followers': user_data.get('followers'),
                        'profile_url': user_data.get('html_url'),
                        'location': user_data.get('location')
                    })
            except requests.exceptions.RequestException as e_user:
                print(f"抓取用戶 {user['login']} 失敗: {e_user}")
        print(f"完成爬取(在 {len(users)} 人中，共有 {len(data)} 人填寫了有效的 Bio。)")
        return pd.DataFrame(data)
    except requests.exceptions.RequestException as e_search:
        if "401" in str(e_search):
             print(f"爬取失敗 (401 Unauthorized)：您的 GITHUB_TOKEN 可能是錯的或已過期。")
        else:
            print(f"爬取失敗：{e_search}")
        return pd.DataFrame()

def write_to_sheet(worksheet_name, df_to_write):
    try:
        spreadsheet = gc.open(GOOGLE_SHEET_NAME)

        try:
            worksheet = spreadsheet.worksheet_by_title(worksheet_name)
        except pygsheets.exceptions.WorksheetNotFound:
            print(f"工作表 '{worksheet_name}' 不存在，正在自動建立...")
            worksheet = spreadsheet.add_worksheet(worksheet_name)

        worksheet.clear()
        worksheet.set_dataframe(df_to_write, start='A1')

        print(f"資料寫入 Google Sheet '{worksheet_name}'！")
        return True

    except Exception as e:
        print(f"寫入失敗: {e}")
        return False

def read_from_sheet(worksheet_name, column_name):
    try:
        spreadsheet = gc.open(GOOGLE_SHEET_NAME)
        worksheet = spreadsheet.worksheet_by_title(worksheet_name)

        df = worksheet.get_as_df()
        if column_name not in df.columns:
            print(f"錯誤：在 '{worksheet_name}' 中找不到名為 '{column_name}' 的欄位。")
            return []

        texts = df[column_name].dropna().astype(str).tolist()
        print(f"成功讀取 {len(texts)} 筆資料。")
        return texts
    except Exception as e:
        print(f"讀取錯誤: {e}")
        return []
def analyze_tfidf_keywords(texts, top_n=5):
    corpus = []
    for text in texts:
        clean_text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9\+#\.]', ' ', text)
        corpus.append(clean_text)
    if not corpus:
        print("沒有可分析的有效文字。")
        return pd.DataFrame()
    vectorizer = TfidfVectorizer(
        stop_words='english',
        token_pattern=r'[a-zA-Z0-9\+#\.]{2,}'
    )
    tfidf_matrix = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names_out()
    scores = tfidf_matrix.sum(axis=0).A1
    df_tfidf = pd.DataFrame({'Keyword': feature_names, 'TF_IDF_Score': scores})
    df_tfidf_sorted = df_tfidf.sort_values(by='TF_IDF_Score', ascending=False)
    df_tfidf_sorted['Keyword'] = df_tfidf_sorted['Keyword'].str.lower()
    df_tfidf_grouped = df_tfidf_sorted.groupby('Keyword')['TF_IDF_Score'].sum().reset_index()
    df_tfidf_final = df_tfidf_grouped.sort_values(by='TF_IDF_Score', ascending=False)
    print(f"✅ TF-IDF 分析完成。")
    return df_tfidf_final.head(top_n)

def generate_insights_for_keyword(selected_keyword, all_keywords_list):
    if not selected_keyword:
        return "請先選擇一個熱詞。", ""

    print(f"🤖 正在呼叫 Gemini API，針對「{selected_keyword}」生成洞察...")
    keywords_str = ", ".join(all_keywords_list)
    prompt = f"""
    你是一位專業的科技業人力資源 (HR) 趨勢分析師。
    我剛才分析了 GitHub 上地點在台灣 (Taiwan) 的開發者個人簡介 (Bio)，得出的 Top 5 技能熱詞是：{keywords_str}
    請你**"只針對「{selected_keyword}」這個詞"**，深入分析它為何成為台灣開發者 Bio 中的 Top 5 熱詞之一，並產生一份精闢的 HR 分析報告。
    請嚴格依照以下格式輸出，不要有任何多餘的文字或開場白：
    ### 洞察摘要
    1. [第一點洞察：解釋「{selected_keyword}」是什麼，為什麼它在技術圈這麼重要？]
    2. [第二點洞察：分析開發者將「{selected_keyword}」寫入 Bio 的動機 (e.g., 求職、展現專業、跟隨趨勢)]
    3. [第三點洞察：從 HR 角度分析，這代表台灣人才市場具備了哪些「{selected_keyword}」相關的技能儲備？]
    4. [第四點洞察：分析這個詞與其他熱詞 (如 {keywords_str}) 之間可能的關聯性]
    5. [第五點洞察：總結企業在招募時，應如何看待具備「{selected_keyword}」技能的候選人]
    ### 結論
    [一段約 120 字的總結，精準說明「{selected_keyword}」這個詞的熱門，反映了台灣技術人才庫的何種趨勢或價值觀。]
    """
    try:
        response = model.generate_content(prompt)
        text_response = response.text.strip()
        summary_part = text_response.split("### 洞察摘要")[1].split("### 結論")[0].strip()
        conclusion_part = text_response.split("### 結論")[1].strip()
        print("✅ Gemini API 已成功生成內容。")
        return summary_part, conclusion_part
    except Exception as e:
        print(f"❌ F_CALL 呼叫 Gemini API 失敗：{e}")
        return f"Gemini API 呼叫失敗: {e}", ""

In [42]:
scraped_df = scrape_github_bios(search_location="Taiwan", num_users=50)

if not scraped_df.empty:
    write_to_sheet("GitHub_Scraped_Data", scraped_df)

    texts_to_analyze = read_from_sheet("GitHub_Scraped_Data", "bio")

    if texts_to_analyze:
        keywords_df = analyze_tfidf_keywords(texts_to_analyze, top_n=5)

        if not keywords_df.empty:
            write_to_sheet("GitHub_Keyword_Stats", keywords_df)
            print("資料準備流程完成")
        else:
            print("分析失敗：未能提取關鍵字。")
    else:
        print("分析失敗：從 Sheet 讀取資料失敗。")
else:
    print("流程終止：爬蟲未能抓取到任何資料。")

成功搜尋到 50 位用戶
完成爬取(在 50 人中，共有 42 人填寫了有效的 Bio。)
資料寫入 Google Sheet 'GitHub_Scraped_Data'！
成功讀取 42 筆資料。
✅ TF-IDF 分析完成。
資料寫入 Google Sheet 'GitHub_Keyword_Stats'！
資料準備流程完成


In [43]:
try:
    top_5_keywords_list = read_from_sheet("GitHub_Keyword_Stats", "Keyword")
    if not top_5_keywords_list:
        print("not top_5_keywords_list")
        top_5_keywords_list = ["(請先執行資料流程)"]
except Exception as e:
    print(f"讀取熱詞失敗: {e}")
    top_5_keywords_list = ["(讀取熱詞失敗)"]

def gradio_get_insights(selected_keyword):
    if not selected_keyword or selected_keyword.startswith("("):
        return "請先選擇一個有效的熱詞。", ""

    summary, conclusion = generate_insights_for_keyword(selected_keyword, top_5_keywords_list)
    return summary, conclusion

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🚀 GitHub 台灣人才趨勢 AI 洞察")
    gr.Markdown(f"本系統已分析完 GitHub 台灣用戶 Bio，目前 Top 5 技能熱詞為：**{', '.join(top_5_keywords_list)}**")

    with gr.Row():
        keyword_dropdown = gr.Dropdown(
            label="請選擇一個技能熱詞進行深入分析",
            choices=top_5_keywords_list
        )

    run_button = gr.Button("🧠 生成 HR 趨勢洞察", variant="primary")

    with gr.Accordion("📊 HR 趨勢分析報告", open=True):
        gr.Markdown("## 💡 AI 洞察摘要")
        summary_output = gr.Textbox(label="", lines=6, interactive=False)
        gr.Markdown("## 📝 AI 結論")
        conclusion_output = gr.Textbox(label="", lines=5, interactive=False)

    run_button.click(
        fn=gradio_get_insights,
        inputs=[keyword_dropdown],
        outputs=[summary_output, conclusion_output]
    )


成功讀取 5 筆資料。


In [ ]:
print("\n" + "="*50)
print("Gradio 介面即將啟動...")
print("請點擊下方顯示的 Public URL (類似 xxx.gradio.live) 來開啟操作介面。")
print("="*50)
demo.launch(debug=True, share=True)


Gradio 介面即將啟動...
請點擊下方顯示的 Public URL (類似 xxx.gradio.live) 來開啟操作介面。
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://abd7d5ec0d1268f533.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🤖 正在呼叫 Gemini API，針對「engineer」生成洞察...
✅ Gemini API 已成功生成內容。
